In [9]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
import pandas as pd

In [2]:
afq_dataset = AFQDataset.from_files(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md", "dki_mk"],
    index_col="subject_id",
    target_cols=["age", "dl_qc_score", "scan_site_id"],
    label_encode_cols=["scan_site_id"]
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1865
(1865, 7200)
(1865, 3)


In [5]:
full_dataset = list(afq_dataset.as_tensorflow_dataset().as_numpy_iterator())

2022-04-07 10:25:49.217761: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
X = np.concatenate([xx[0][None] for xx in full_dataset], 0)
y = np.array([yy[1][0] for yy in full_dataset])
qc = np.array([yy[1][1] for yy in full_dataset])
site = np.array([yy[1][2] for yy in full_dataset])

In [7]:
X = X[qc>0]
y = y[qc>0]
site = site[qc>0]

In [8]:
# # Split the data by sites
# X_0 = X[site==0]
# y_0 = y[site==0]
# X_3 = X[site==3]
# y_3 = y[site==3]
# X_4 = X[site==4]
# y_4 = y[site==4]

In [9]:
# # Split the data into train and test sets:
# X_0_train, X_0_test, y_0_train, y_0_test = train_test_split(X_0, y_0, test_size=0.2, random_state=42)
# X_3_train, X_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, test_size=0.2, random_state=42)
# X_4_train, X_4_test, y_4_train, y_4_test = train_test_split(X_4, y_4, test_size=0.2, random_state=42)

In [10]:
# imputer = SimpleImputer(strategy="median")

In [11]:
# # Impute train and test separately:
# X_0_train = np.concatenate([imputer.fit_transform(X_0_train[..., ii])[:, :, None] for ii in range(X_0_train.shape[-1])], -1)
# X_0_test = np.concatenate([imputer.fit_transform(X_0_test[..., ii])[:, :, None] for ii in range(X_0_test.shape[-1])], -1)
# X_3_train = np.concatenate([imputer.fit_transform(X_3_train[..., ii])[:, :, None] for ii in range(X_3_train.shape[-1])], -1)
# X_3_test = np.concatenate([imputer.fit_transform(X_3_test[..., ii])[:, :, None] for ii in range(X_3_test.shape[-1])], -1)
# X_4_train = np.concatenate([imputer.fit_transform(X_4_train[..., ii])[:, :, None] for ii in range(X_4_train.shape[-1])], -1)
# X_4_test = np.concatenate([imputer.fit_transform(X_4_test[..., ii])[:, :, None] for ii in range(X_4_test.shape[-1])], -1)

In [8]:
n_epochs = 1000

# EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    mode="min",
    patience=100
)

# ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=20,
    verbose=1,
)

In [30]:
# Generate evaluation results and correlation coeffcients combined in a dataframe, and history
def single_cross_site(model_name, name_str, ckpt_filepath, lr,
                      site_1, site_2, site_3, X, y):
    # Split the data by sites
    X_1 = X[site==site_1]
    y_1 = y[site==site_1]
    X_2 = X[site==site_2]
    y_2 = y[site==site_2]
    X_3 = X[site==site_3]
    y_3 = y[site==site_3]
    # Split the data into train and test sets:
    X_train, X_test1, y_train, y_test1 = train_test_split(X_1, y_1, test_size=0.2)
    _, X_test2, _, y_test2 = train_test_split(X_2, y_2, test_size=0.2)
    _, X_test3, _, y_test3 = train_test_split(X_3, y_3, test_size=0.2)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test1 = np.concatenate([imputer.fit_transform(X_test1[..., ii])[:, :, None] for ii in range(X_test1.shape[-1])], -1)
    X_test2 = np.concatenate([imputer.fit_transform(X_test2[..., ii])[:, :, None] for ii in range(X_test2.shape[-1])], -1)
    X_test3 = np.concatenate([imputer.fit_transform(X_test3[..., ii])[:, :, None] for ii in range(X_test3.shape[-1])], -1)
    model = model_name(input_shape=(100, 72), n_classes=1, output_activation=None, verbose=True)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['mean_squared_error', 
                           tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                           'mean_absolute_error'])
    # ModelCheckpoint
    ckpt = tf.keras.callbacks.ModelCheckpoint(
    ckpt_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    )
    # CSVLogger
    log = tf.keras.callbacks.CSVLogger(filename=(str(model_name) + '.csv'), append=True)
    callbacks = [early_stopping, ckpt, reduce_lr, log]
    history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=128, validation_split=0.2,
                        callbacks=callbacks)
    model.load_weights(ckpt_filepath)
    y_predicted1 = model.predict(X_test1)
    y_predicted1 = y_predicted1.reshape(y_test1.shape)
    y_predicted2 = model.predict(X_test2)
    y_predicted2 = y_predicted2.reshape(y_test2.shape)
    y_predicted3 = model.predict(X_test3)
    y_predicted3 = y_predicted3.reshape(y_test3.shape)
    coef1 = np.corrcoef(y_test1, y_predicted1)[0,1] ** 2
    coef2 = np.corrcoef(y_test2, y_predicted2)[0,1] ** 2
    coef3 = np.corrcoef(y_test3, y_predicted3)[0,1] ** 2
    eval_1 = model.evaluate(X_test1, y_test1)
    eval_2 = model.evaluate(X_test2, y_test2)
    eval_3 = model.evaluate(X_test3, y_test3)
    result = {'Model': [name_str]*12,
              'Train_site': [site_1]*12,
              'Test_site': [site_1] * 4 + [site_2] * 4 + [site_3] * 4,
              'Metric': ['MSE', 'RMSE', 'MAE', 'coef'] * 3,
              'Value': [eval_1[1], eval_1[2], eval_1[3], coef1,
                        eval_2[1], eval_2[2], eval_2[3], coef2,
                        eval_3[1], eval_3[2], eval_3[3], coef3]}
    df = pd.DataFrame(result)
    return df, history

In [31]:
df_lenet1, history_lenet1 = single_cross_site(cnn_lenet, 'cnn_lenet', '/tmp/lenet.m5', 0.001, 0, 3, 4, X, y)

pooling layers: 4
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_16 (Conv1D)          (None, 100, 6)            1302      
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 50, 6)            0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 50, 16)            304       
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 25, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 25, 2

Epoch 13/1000
3/4 [=====================>........] - ETA: 0s - loss: 19.9975 - mean_squared_error: 19.9975 - rmse: 4.4719 - mean_absolute_error: 3.5199
Epoch 00013: val_loss did not improve from 10.32761
4/4 [==============================] - 0s 38ms/step - loss: 19.2346 - mean_squared_error: 19.2346 - rmse: 4.3857 - mean_absolute_error: 3.4381 - val_loss: 13.4040 - val_mean_squared_error: 13.4040 - val_rmse: 3.6611 - val_mean_absolute_error: 2.7109 - lr: 0.0010
Epoch 14/1000
4/4 [==============================] - ETA: 0s - loss: 19.1802 - mean_squared_error: 19.1802 - rmse: 4.3795 - mean_absolute_error: 3.4263
Epoch 00014: val_loss did not improve from 10.32761
4/4 [==============================] - 0s 25ms/step - loss: 19.1802 - mean_squared_error: 19.1802 - rmse: 4.3795 - mean_absolute_error: 3.4263 - val_loss: 12.9031 - val_mean_squared_error: 12.9031 - val_rmse: 3.5921 - val_mean_absolute_error: 2.6604 - lr: 0.0010
Epoch 15/1000
1/4 [======>.......................] - ETA: 0s - los

Epoch 31/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.8748 - mean_squared_error: 15.8748 - rmse: 3.9843 - mean_absolute_error: 3.1994
Epoch 00031: val_loss did not improve from 10.32761
4/4 [==============================] - 0s 23ms/step - loss: 15.6499 - mean_squared_error: 15.6499 - rmse: 3.9560 - mean_absolute_error: 3.1092 - val_loss: 11.4605 - val_mean_squared_error: 11.4605 - val_rmse: 3.3853 - val_mean_absolute_error: 2.5374 - lr: 5.0000e-04
Epoch 32/1000
1/4 [======>.......................] - ETA: 0s - loss: 19.8109 - mean_squared_error: 19.8109 - rmse: 4.4509 - mean_absolute_error: 3.3150
Epoch 00032: val_loss did not improve from 10.32761
4/4 [==============================] - 0s 22ms/step - loss: 17.7837 - mean_squared_error: 17.7837 - rmse: 4.2171 - mean_absolute_error: 3.2769 - val_loss: 10.7237 - val_mean_squared_error: 10.7237 - val_rmse: 3.2747 - val_mean_absolute_error: 2.5080 - lr: 5.0000e-04
Epoch 33/1000
1/4 [======>.......................] - ETA: 

4/4 [==============================] - ETA: 0s - loss: 16.2040 - mean_squared_error: 16.2040 - rmse: 4.0254 - mean_absolute_error: 3.2024
Epoch 00048: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 28ms/step - loss: 16.2040 - mean_squared_error: 16.2040 - rmse: 4.0254 - mean_absolute_error: 3.2024 - val_loss: 11.3588 - val_mean_squared_error: 11.3588 - val_rmse: 3.3703 - val_mean_absolute_error: 2.5296 - lr: 5.0000e-04
Epoch 49/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.3402 - mean_squared_error: 15.3402 - rmse: 3.9167 - mean_absolute_error: 3.0691
Epoch 00049: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 23ms/step - loss: 16.1891 - mean_squared_error: 16.1891 - rmse: 4.0236 - mean_absolute_error: 3.2110 - val_loss: 11.1132 - val_mean_squared_error: 11.1132 - val_rmse: 3.3337 - val_mean_absolute_error: 2.5182 - lr: 5.0000e-04
Epoch 50/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.

Epoch 66/1000
4/4 [==============================] - ETA: 0s - loss: 17.0958 - mean_squared_error: 17.0958 - rmse: 4.1347 - mean_absolute_error: 3.2673
Epoch 00066: val_loss did not improve from 10.16643

Epoch 00066: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 26ms/step - loss: 17.0958 - mean_squared_error: 17.0958 - rmse: 4.1347 - mean_absolute_error: 3.2673 - val_loss: 11.1002 - val_mean_squared_error: 11.1002 - val_rmse: 3.3317 - val_mean_absolute_error: 2.5136 - lr: 5.0000e-04
Epoch 67/1000
1/4 [======>.......................] - ETA: 0s - loss: 16.3999 - mean_squared_error: 16.3999 - rmse: 4.0497 - mean_absolute_error: 3.0380
Epoch 00067: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 24ms/step - loss: 14.3302 - mean_squared_error: 14.3302 - rmse: 3.7855 - mean_absolute_error: 2.9665 - val_loss: 10.5447 - val_mean_squared_error: 10.5447 - val_rmse: 3.2473 - val_mean_absolute_error: 2

1/4 [======>.......................] - ETA: 0s - loss: 13.3855 - mean_squared_error: 13.3855 - rmse: 3.6586 - mean_absolute_error: 2.8160
Epoch 00083: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 24ms/step - loss: 14.9379 - mean_squared_error: 14.9379 - rmse: 3.8650 - mean_absolute_error: 2.9636 - val_loss: 11.0193 - val_mean_squared_error: 11.0193 - val_rmse: 3.3195 - val_mean_absolute_error: 2.5066 - lr: 2.5000e-04
Epoch 84/1000
1/4 [======>.......................] - ETA: 0s - loss: 14.8828 - mean_squared_error: 14.8828 - rmse: 3.8578 - mean_absolute_error: 3.0050
Epoch 00084: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 23ms/step - loss: 14.2847 - mean_squared_error: 14.2847 - rmse: 3.7795 - mean_absolute_error: 2.9417 - val_loss: 10.5267 - val_mean_squared_error: 10.5267 - val_rmse: 3.2445 - val_mean_absolute_error: 2.4894 - lr: 2.5000e-04
Epoch 85/1000
1/4 [======>.......................] - ETA: 0s - loss: 17.

1/4 [======>.......................] - ETA: 0s - loss: 12.6963 - mean_squared_error: 12.6963 - rmse: 3.5632 - mean_absolute_error: 2.7185
Epoch 00100: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 22ms/step - loss: 14.8188 - mean_squared_error: 14.8188 - rmse: 3.8495 - mean_absolute_error: 3.0407 - val_loss: 10.6364 - val_mean_squared_error: 10.6364 - val_rmse: 3.2613 - val_mean_absolute_error: 2.4909 - lr: 1.2500e-04
Epoch 101/1000
4/4 [==============================] - ETA: 0s - loss: 15.8131 - mean_squared_error: 15.8131 - rmse: 3.9766 - mean_absolute_error: 3.1184
Epoch 00101: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 25ms/step - loss: 15.8131 - mean_squared_error: 15.8131 - rmse: 3.9766 - mean_absolute_error: 3.1184 - val_loss: 10.4976 - val_mean_squared_error: 10.4976 - val_rmse: 3.2400 - val_mean_absolute_error: 2.4861 - lr: 1.2500e-04
Epoch 102/1000
4/4 [==============================] - ETA: 0s - loss: 1

4/4 [==============================] - ETA: 0s - loss: 14.5130 - mean_squared_error: 14.5130 - rmse: 3.8096 - mean_absolute_error: 3.0688
Epoch 00117: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 25ms/step - loss: 14.5130 - mean_squared_error: 14.5130 - rmse: 3.8096 - mean_absolute_error: 3.0688 - val_loss: 10.6056 - val_mean_squared_error: 10.6056 - val_rmse: 3.2566 - val_mean_absolute_error: 2.4870 - lr: 6.2500e-05
Epoch 118/1000
1/4 [======>.......................] - ETA: 0s - loss: 16.5867 - mean_squared_error: 16.5867 - rmse: 4.0727 - mean_absolute_error: 3.2977
Epoch 00118: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 24ms/step - loss: 15.3770 - mean_squared_error: 15.3770 - rmse: 3.9214 - mean_absolute_error: 3.1074 - val_loss: 10.6008 - val_mean_squared_error: 10.6008 - val_rmse: 3.2559 - val_mean_absolute_error: 2.4868 - lr: 6.2500e-05
Epoch 119/1000
1/4 [======>.......................] - ETA: 0s - loss: 1

1/4 [======>.......................] - ETA: 0s - loss: 14.7398 - mean_squared_error: 14.7398 - rmse: 3.8392 - mean_absolute_error: 2.9759
Epoch 00134: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 23ms/step - loss: 15.2385 - mean_squared_error: 15.2385 - rmse: 3.9037 - mean_absolute_error: 3.0815 - val_loss: 10.6607 - val_mean_squared_error: 10.6607 - val_rmse: 3.2651 - val_mean_absolute_error: 2.4874 - lr: 3.1250e-05
Epoch 135/1000
1/4 [======>.......................] - ETA: 0s - loss: 17.6562 - mean_squared_error: 17.6562 - rmse: 4.2019 - mean_absolute_error: 3.1953
Epoch 00135: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 22ms/step - loss: 14.5761 - mean_squared_error: 14.5761 - rmse: 3.8179 - mean_absolute_error: 2.9782 - val_loss: 10.6400 - val_mean_squared_error: 10.6400 - val_rmse: 3.2619 - val_mean_absolute_error: 2.4867 - lr: 3.1250e-05
Epoch 136/1000
1/4 [======>.......................] - ETA: 0s - loss: 1

In [33]:
df_lenet2, history_lenet2 = single_cross_site(cnn_lenet, 'cnn_lenet', '/tmp/lenet.m5', 0.001, 3, 0, 4, X, y)

pooling layers: 4
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_20 (Conv1D)          (None, 100, 6)            1302      
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 50, 6)            0         
 g1D)                                                            
                                                                 
 conv1d_21 (Conv1D)          (None, 50, 16)            304       
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, 25, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_22 (Conv1D)          (None, 25, 2

Epoch 13/1000
1/4 [======>.......................] - ETA: 0s - loss: 19.1181 - mean_squared_error: 19.1181 - rmse: 4.3724 - mean_absolute_error: 3.3425
Epoch 00013: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 23ms/step - loss: 18.1769 - mean_squared_error: 18.1769 - rmse: 4.2634 - mean_absolute_error: 3.3002 - val_loss: 13.2816 - val_mean_squared_error: 13.2816 - val_rmse: 3.6444 - val_mean_absolute_error: 2.6690 - lr: 0.0010
Epoch 14/1000
1/4 [======>.......................] - ETA: 0s - loss: 21.5543 - mean_squared_error: 21.5543 - rmse: 4.6427 - mean_absolute_error: 3.7119
Epoch 00014: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 24ms/step - loss: 17.5411 - mean_squared_error: 17.5411 - rmse: 4.1882 - mean_absolute_error: 3.2593 - val_loss: 13.7641 - val_mean_squared_error: 13.7641 - val_rmse: 3.7100 - val_mean_absolute_error: 2.6981 - lr: 0.0010
Epoch 15/1000
1/4 [======>.......................] - ETA: 0s - los

Epoch 31/1000
1/4 [======>.......................] - ETA: 0s - loss: 14.0081 - mean_squared_error: 14.0081 - rmse: 3.7427 - mean_absolute_error: 3.0074
Epoch 00031: val_loss did not improve from 11.71418

Epoch 00031: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 21ms/step - loss: 16.2056 - mean_squared_error: 16.2056 - rmse: 4.0256 - mean_absolute_error: 3.2271 - val_loss: 12.0130 - val_mean_squared_error: 12.0130 - val_rmse: 3.4660 - val_mean_absolute_error: 2.6276 - lr: 0.0010
Epoch 32/1000
1/4 [======>.......................] - ETA: 0s - loss: 14.3218 - mean_squared_error: 14.3218 - rmse: 3.7844 - mean_absolute_error: 2.9411
Epoch 00032: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 20ms/step - loss: 16.5577 - mean_squared_error: 16.5577 - rmse: 4.0691 - mean_absolute_error: 3.1655 - val_loss: 12.3184 - val_mean_squared_error: 12.3184 - val_rmse: 3.5098 - val_mean_absolute_error: 2.626

1/4 [======>.......................] - ETA: 0s - loss: 16.4013 - mean_squared_error: 16.4013 - rmse: 4.0499 - mean_absolute_error: 3.0686
Epoch 00048: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 14.0233 - mean_squared_error: 14.0233 - rmse: 3.7448 - mean_absolute_error: 2.9217 - val_loss: 12.3691 - val_mean_squared_error: 12.3691 - val_rmse: 3.5170 - val_mean_absolute_error: 2.6244 - lr: 5.0000e-04
Epoch 49/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.8861 - mean_squared_error: 15.8861 - rmse: 3.9857 - mean_absolute_error: 3.0447
Epoch 00049: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 16.2467 - mean_squared_error: 16.2467 - rmse: 4.0307 - mean_absolute_error: 3.1404 - val_loss: 12.2834 - val_mean_squared_error: 12.2834 - val_rmse: 3.5048 - val_mean_absolute_error: 2.6169 - lr: 5.0000e-04
Epoch 50/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.

1/4 [======>.......................] - ETA: 0s - loss: 12.6064 - mean_squared_error: 12.6064 - rmse: 3.5505 - mean_absolute_error: 2.7665
Epoch 00065: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 23ms/step - loss: 14.7416 - mean_squared_error: 14.7416 - rmse: 3.8395 - mean_absolute_error: 2.9773 - val_loss: 12.5551 - val_mean_squared_error: 12.5551 - val_rmse: 3.5433 - val_mean_absolute_error: 2.6212 - lr: 2.5000e-04
Epoch 66/1000
1/4 [======>.......................] - ETA: 0s - loss: 14.2518 - mean_squared_error: 14.2518 - rmse: 3.7752 - mean_absolute_error: 2.9055
Epoch 00066: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 16.1382 - mean_squared_error: 16.1382 - rmse: 4.0172 - mean_absolute_error: 3.0608 - val_loss: 12.2891 - val_mean_squared_error: 12.2891 - val_rmse: 3.5056 - val_mean_absolute_error: 2.6171 - lr: 2.5000e-04
Epoch 67/1000
1/4 [======>.......................] - ETA: 0s - loss: 13.

1/4 [======>.......................] - ETA: 0s - loss: 12.4869 - mean_squared_error: 12.4869 - rmse: 3.5337 - mean_absolute_error: 2.8318
Epoch 00082: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 23ms/step - loss: 13.6250 - mean_squared_error: 13.6250 - rmse: 3.6912 - mean_absolute_error: 2.9073 - val_loss: 11.8878 - val_mean_squared_error: 11.8878 - val_rmse: 3.4479 - val_mean_absolute_error: 2.6316 - lr: 1.2500e-04
Epoch 83/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.4968 - mean_squared_error: 15.4968 - rmse: 3.9366 - mean_absolute_error: 2.9420
Epoch 00083: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 14.4205 - mean_squared_error: 14.4205 - rmse: 3.7974 - mean_absolute_error: 2.9454 - val_loss: 11.8128 - val_mean_squared_error: 11.8128 - val_rmse: 3.4370 - val_mean_absolute_error: 2.6366 - lr: 1.2500e-04
Epoch 84/1000
1/4 [======>.......................] - ETA: 0s - loss: 17.

1/4 [======>.......................] - ETA: 0s - loss: 10.5593 - mean_squared_error: 10.5593 - rmse: 3.2495 - mean_absolute_error: 2.5910
Epoch 00099: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 14.4834 - mean_squared_error: 14.4834 - rmse: 3.8057 - mean_absolute_error: 2.9227 - val_loss: 12.0558 - val_mean_squared_error: 12.0558 - val_rmse: 3.4721 - val_mean_absolute_error: 2.6072 - lr: 6.2500e-05
Epoch 100/1000
1/4 [======>.......................] - ETA: 0s - loss: 13.8960 - mean_squared_error: 13.8960 - rmse: 3.7277 - mean_absolute_error: 2.8233
Epoch 00100: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 13.6119 - mean_squared_error: 13.6119 - rmse: 3.6894 - mean_absolute_error: 2.9083 - val_loss: 12.0085 - val_mean_squared_error: 12.0085 - val_rmse: 3.4653 - val_mean_absolute_error: 2.6082 - lr: 6.2500e-05
Epoch 101/1000
1/4 [======>.......................] - ETA: 0s - loss: 1

In [35]:
df_lenet3, history_lenet3 = single_cross_site(cnn_lenet, 'cnn_lenet', '/tmp/lenet.m5', 0.001, 4, 0, 3, X, y)

pooling layers: 4
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_24 (Conv1D)          (None, 100, 6)            1302      
                                                                 
 max_pooling1d_24 (MaxPoolin  (None, 50, 6)            0         
 g1D)                                                            
                                                                 
 conv1d_25 (Conv1D)          (None, 50, 16)            304       
                                                                 
 max_pooling1d_25 (MaxPoolin  (None, 25, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_26 (Conv1D)          (None, 25, 2

1/2 [==============>...............] - ETA: 0s - loss: 31.7388 - mean_squared_error: 31.7388 - rmse: 5.6337 - mean_absolute_error: 4.3412
Epoch 00012: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 31ms/step - loss: 32.0512 - mean_squared_error: 32.0512 - rmse: 5.6614 - mean_absolute_error: 4.4322 - val_loss: 31.0547 - val_mean_squared_error: 31.0547 - val_rmse: 5.5727 - val_mean_absolute_error: 4.4722 - lr: 0.0010
Epoch 13/1000
1/2 [==============>...............] - ETA: 0s - loss: 28.2674 - mean_squared_error: 28.2674 - rmse: 5.3167 - mean_absolute_error: 4.2358
Epoch 00013: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 30ms/step - loss: 27.7825 - mean_squared_error: 27.7825 - rmse: 5.2709 - mean_absolute_error: 4.1993 - val_loss: 22.7370 - val_mean_squared_error: 22.7370 - val_rmse: 4.7683 - val_mean_absolute_error: 3.8863 - lr: 0.0010
Epoch 14/1000
1/2 [==============>...............] - ETA: 0s - loss: 28.1706 - m

Epoch 30/1000
1/2 [==============>...............] - ETA: 0s - loss: 25.0847 - mean_squared_error: 25.0847 - rmse: 5.0085 - mean_absolute_error: 3.8657
Epoch 00030: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 32ms/step - loss: 25.1820 - mean_squared_error: 25.1820 - rmse: 5.0182 - mean_absolute_error: 3.9221 - val_loss: 21.4500 - val_mean_squared_error: 21.4500 - val_rmse: 4.6314 - val_mean_absolute_error: 3.7650 - lr: 5.0000e-04
Epoch 31/1000
1/2 [==============>...............] - ETA: 0s - loss: 22.8326 - mean_squared_error: 22.8326 - rmse: 4.7784 - mean_absolute_error: 3.9537
Epoch 00031: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 31ms/step - loss: 22.3744 - mean_squared_error: 22.3744 - rmse: 4.7302 - mean_absolute_error: 3.9068 - val_loss: 21.6073 - val_mean_squared_error: 21.6073 - val_rmse: 4.6484 - val_mean_absolute_error: 3.7765 - lr: 5.0000e-04
Epoch 32/1000
1/2 [==============>...............] - ETA: 

Epoch 48/1000
1/2 [==============>...............] - ETA: 0s - loss: 16.1689 - mean_squared_error: 16.1689 - rmse: 4.0211 - mean_absolute_error: 3.2214
Epoch 00048: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 32ms/step - loss: 15.9391 - mean_squared_error: 15.9391 - rmse: 3.9924 - mean_absolute_error: 3.1827 - val_loss: 20.0891 - val_mean_squared_error: 20.0891 - val_rmse: 4.4821 - val_mean_absolute_error: 3.6330 - lr: 2.5000e-04
Epoch 49/1000
1/2 [==============>...............] - ETA: 0s - loss: 15.8018 - mean_squared_error: 15.8018 - rmse: 3.9751 - mean_absolute_error: 3.2155
Epoch 00049: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 30ms/step - loss: 15.7096 - mean_squared_error: 15.7096 - rmse: 3.9635 - mean_absolute_error: 3.2210 - val_loss: 19.4771 - val_mean_squared_error: 19.4771 - val_rmse: 4.4133 - val_mean_absolute_error: 3.5863 - lr: 2.5000e-04
Epoch 50/1000
1/2 [==============>...............] - ETA: 

2/2 [==============================] - 0s 29ms/step - loss: 18.4533 - mean_squared_error: 18.4533 - rmse: 4.2957 - mean_absolute_error: 3.2805 - val_loss: 18.3528 - val_mean_squared_error: 18.3528 - val_rmse: 4.2840 - val_mean_absolute_error: 3.4734 - lr: 2.5000e-04
Epoch 66/1000
1/2 [==============>...............] - ETA: 0s - loss: 16.4594 - mean_squared_error: 16.4594 - rmse: 4.0570 - mean_absolute_error: 3.1964
Epoch 00066: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 29ms/step - loss: 16.1955 - mean_squared_error: 16.1955 - rmse: 4.0244 - mean_absolute_error: 3.1848 - val_loss: 18.5533 - val_mean_squared_error: 18.5533 - val_rmse: 4.3074 - val_mean_absolute_error: 3.4861 - lr: 2.5000e-04
Epoch 67/1000
1/2 [==============>...............] - ETA: 0s - loss: 15.9205 - mean_squared_error: 15.9205 - rmse: 3.9901 - mean_absolute_error: 3.0911
Epoch 00067: val_loss did not improve from 12.31365

Epoch 00067: ReduceLROnPlateau reducing learning rate to 

Epoch 83/1000
1/2 [==============>...............] - ETA: 0s - loss: 17.3410 - mean_squared_error: 17.3410 - rmse: 4.1643 - mean_absolute_error: 3.2600
Epoch 00083: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 30ms/step - loss: 20.2177 - mean_squared_error: 20.2177 - rmse: 4.4964 - mean_absolute_error: 3.5360 - val_loss: 18.3929 - val_mean_squared_error: 18.3929 - val_rmse: 4.2887 - val_mean_absolute_error: 3.4394 - lr: 1.2500e-04
Epoch 84/1000
1/2 [==============>...............] - ETA: 0s - loss: 17.8778 - mean_squared_error: 17.8778 - rmse: 4.2282 - mean_absolute_error: 3.4828
Epoch 00084: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 29ms/step - loss: 17.2060 - mean_squared_error: 17.2060 - rmse: 4.1480 - mean_absolute_error: 3.4649 - val_loss: 18.0208 - val_mean_squared_error: 18.0208 - val_rmse: 4.2451 - val_mean_absolute_error: 3.4067 - lr: 1.2500e-04
Epoch 85/1000
1/2 [==============>...............] - ETA: 

1/2 [==============>...............] - ETA: 0s - loss: 19.1628 - mean_squared_error: 19.1628 - rmse: 4.3775 - mean_absolute_error: 3.3959
Epoch 00100: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 29ms/step - loss: 19.5406 - mean_squared_error: 19.5406 - rmse: 4.4205 - mean_absolute_error: 3.4260 - val_loss: 18.6803 - val_mean_squared_error: 18.6803 - val_rmse: 4.3221 - val_mean_absolute_error: 3.4540 - lr: 6.2500e-05
Epoch 101/1000
1/2 [==============>...............] - ETA: 0s - loss: 18.4199 - mean_squared_error: 18.4199 - rmse: 4.2918 - mean_absolute_error: 3.4061
Epoch 00101: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 27ms/step - loss: 17.1577 - mean_squared_error: 17.1577 - rmse: 4.1422 - mean_absolute_error: 3.2653 - val_loss: 17.9510 - val_mean_squared_error: 17.9510 - val_rmse: 4.2369 - val_mean_absolute_error: 3.3935 - lr: 6.2500e-05
Epoch 102/1000
1/2 [==============>...............] - ETA: 0s - loss: 1

In [45]:
df_lenet = (df_lenet1.merge(df_lenet2, how='outer')).merge(df_lenet3, how='outer')

In [157]:
def to_arr(dict):
    lists = dict.items()
    x,y = zip(*lists)
    arr = np.array(y)
    return arr

In [171]:
to_arr(results_1_3)[0,:]          

array([11.16009808,  3.34067321,  2.61717224])